## Final Project Part 3

<b>Syeda Rija Zaineb</b>

In [1]:
myJekyllSaveDir = '/Users/rijaz/szaineb.github.io/assets/json/'

In [2]:
import altair as alt
import requests
import pandas as pd
import geopandas
import json

<b>Contextual Visualizations - Chicago Hate Crimes</b>

In [3]:
chicago_crimes = pd.read_csv("https://raw.githubusercontent.com/szaineb/szaineb.github.io/master/data/ChicagoData.csv")

In [4]:
chicago_crimes

,RD,DATE,TIME,UCR DODE,OFFENSE,Motivation,CPD Area,DIST,BEAT,Community,Victims,Arrested,Adults,Juveniles,Disposition
0,HV113109,1/10/2012 0:00,2100,1320,CRIMINAL DAMAGE: TO VEHICLE,ANTI-JEWISH,Central,12.0,1212.0,WEST TOWN,1.0,0.0,NaN,NaN,UNDETERMINED
1,HV117394,1/12/2012 0:00,2359,580,STALKING: SIMPLE,ANTI-MULTIPLE RELIGIONS/GROUPS,Central,18.0,1822.0,NEAR NORTH SIDE,1.0,0.0,NaN,NaN,UNFOUNDED
2,HV119102,1/16/2012 0:00,1620,320,ROBBERY: STRONGARM - NO WEAPON,ANTI-ASIAN,Central,9.0,914.0,ARMOUR SQUARE,1.0,7.0,1.0,6.0,UNFOUNDED
3,HV132973,1/26/2012 0:00,1500,1305,CRIMINAL DAMAGE: CRIMINAL DEFACEMENT,ANTI-GAY (MALE),Central,18.0,1812.0,LINCOLN PARK,1.0,0.0,NaN,NaN,UNDETERMINED
4,HV139773,1/31/2012 0:00,1356,2825,OTHER OFFENSE: HARASSMENT BY TELEPHONE,ANTI-BLACK/AFRICAN-AMERICAN,Central,1.0,133.0,NEAR SOUTH SIDE,1.0,0.0,NaN,NaN,UNFOUNDED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,JF435363,10/15/2022 0:00,347,1310,CRIMINAL DAMAGE: TO PROPERTY,ANTI-GAY/LESBIAN/BISEXUAL/TRANSGENDER (MIXED ...,North,16.0,1622.0,JEFFERSON PARK,1.0,0.0,NaN,NaN,BONA FIDE
909,JF439202,10/18/2022 0:00,610,560,ASSAULT: SIMPLE,ANTI-GAY (MALE),North,24.0,2422.0,ROGERS PARK,1.0,0.0,NaN,NaN,BONA FIDE
910,JF440861,10/18/2022 0:00,1800,1305,CRIMINAL DAMAGE: CRIMINAL DEFACEMENT,ANTI-JEWISH,North,19.0,1932.0,LAKE VIEW,1.0,0.0,NaN,NaN,BONA FIDE
911,JF446242,10/23/2022 0:00,10,051a,ASSAULT: AGGRAVATED: HANDGUN,ANTI-BLACK/AFRICAN-AMERICAN,North,25.0,2514.0,BELMONT CRAGIN,1.0,0.0,NaN,NaN,BONA FIDE


In [5]:
chicago_crimes['Year'] = pd.to_datetime(chicago_crimes['DATE']).dt.year

In [6]:
# chicago_crimes['Year']

In [7]:
chicago_crimes_2021 = chicago_crimes[chicago_crimes['Year'] == 2021]

In [8]:
# chicago_crimes_2021.groupby('Community').count()

<b>Top 5 Bias Motives in Chicago 2021</b>

In [9]:
bar_base_chicago = alt.Chart(chicago_crimes_2021).mark_bar().encode(
    x=alt.X('count:Q', axis=None),
    y=alt.Y('Motivation:N',
        sort=alt.EncodingSortField(field='count', order='descending', op='sum'), axis=alt.Axis(title=None)
    ),
    tooltip=alt.Tooltip('count:Q')
).transform_aggregate(
    count='count()',
    groupby=['Motivation']
).transform_window(
    window=[{'op': 'rank', 'as': 'rank'}],
    sort=[{'field': 'count', 'order': 'descending'}]
).transform_filter('datum.rank <= 5'
).properties(title="Top 5 Motives behind Hate Crimes in Chicago", width=550, height=100)

color_bar_chicago = bar_base_chicago.mark_bar().encode(
    color=alt.Color('Complaint Year Number:O', scale=alt.Scale(scheme='teals'),legend=None),
    opacity=alt.value(0.7)
)

bar_label_chicago = bar_base_chicago.mark_text(
    align='right',
    baseline='middle',
    dx=18,
).encode(
    text='count:Q'
)

bar_bias_chicago = color_bar_chicago + bar_label_chicago

In [10]:
bar_bias_chicago

alt.LayerChart(...)

In [11]:
bar_bias_chicago.save(myJekyllSaveDir+'Chicago_Top5_Motives.json')

In [12]:
bar_base_chicago_boroughs = alt.Chart(chicago_crimes_2021).mark_bar().encode(
    x=alt.X('count:Q', axis=None),
    y=alt.Y('Community:N',
        sort=alt.EncodingSortField(field='count', order='descending', op='sum'), axis=alt.Axis(title=None)
    ),
    tooltip=alt.Tooltip('count:Q')
).transform_aggregate(
    count='count()',
    groupby=['Community']
).transform_window(
    window=[{'op': 'rank', 'as': 'rank'}],
    sort=[{'field': 'count', 'order': 'descending'}]
).transform_filter('datum.rank <= 5'
).properties(title="Top 5 Communities with most Hate Crimes in Chicago", width=550, height=100)

color_bar_chicago_boroughs = bar_base_chicago_boroughs.mark_bar().encode(
    color=alt.Color('Complaint Year Number:O', scale=alt.Scale(scheme='teals'),legend=None),
    opacity=alt.value(0.7)
)

bar_label_chicago_boroughs = bar_base_chicago_boroughs.mark_text(
    align='right',
    baseline='middle',
    dx=18,
).encode(
    text='count:Q'
)

bar_bias_chicago_communities = color_bar_chicago_boroughs + bar_label_chicago_boroughs

In [13]:
bar_bias_chicago_communities

alt.LayerChart(...)

In [14]:
bar_bias_chicago_communities.save(myJekyllSaveDir+'Chicago_Top5_Communities.json')

<b>Write-up</b>

In this notebook, I have developed all the visualizations published in my article <a href="https://szaineb.github.io/projects/final-project">here</a>.

I used public data from New York Police Department (NYPD) and Chicago Police Department (CPD) to create all these visualizations. I have uploaded all these data sets on my github repository and also added a link to them in my article.

In the description here and in the article, I have interchangeably used complaints filed and hate crimes terms to represent the same thing.

I have first developed 2 bar charts using the merged data from two datasets 'Police_Precincts.geojson' and 'NYPD Hate Crimes.csv'. In the first chart, I presented Top 5 bias motivations for hate crimes in NYC. In the second chart, I showed the patrol borough sorted by count of hate crime incidents. I used bar chart for as I am comparing categorical variables here.

Then I have an interactive dashboard which gives an overview of the hate crime complaints filed in various NYC police precincts during the year 2021. By using this dashboard, users can see the number of complaints filed across precincts, the areas that had the least and most crimes as well as the biases that motivated hate crimes in NYC in the year 2021.

This dashboard has two charts. On the left you can find the NYC map based on police precincts. The map is color coded by the number of hate crime complaints filed in 2021. I used yelloworangered color scheme for this. Lighter yellow shade shows less number of crimes while brightest red color depicts highest number of crime complaints. On the right, there is a bar chart that shows count of complaints by 'Bias Motive Description'. If we look at the overall stats, we see that majority of the crimes had anti-jewish motives followed by anti-asian.

For contextual visualizations, I chose to analyze Chicago hate crimes as NYC and Chicago city are often compared for crime rates. I used 'ChicagoData.csv' that I got from Chicago Police Department's website. I created 2 bar charts using this file. The first chart displays top 5 bias motivations for hate crimes in Chicago and the second one shows top 5 communities with most incidents.

I have tried to follow a consitent design theme. I used similar charts so that comparison is easy to understand. Similarly, I followed orange, red theme for NYC as we are talking about crimes so red seemed appropriate. I followed same color scheme for all NYC charts to ensure that it is easy for users to follow that all this information is some how related. To differentiate Chicago from NYC, I used teals color scheme for its charts.

<b>Data Sources</b>

Police Precincts. (n.d.). NYC Open Data. https://data.cityofnewyork.us/Public-Safety/Police-Precincts/78dh-3ptz

NYPD Hate Crimes | NYC Open Data. (2022, October 19). https://data.cityofnewyork.us/Public-Safety/NYPD-Hate-Crimes/bqiq-cu78

Hate Crime Dashboard. (n.d.). Chicago Police Department. https://home.chicagopolice.org/statistics-data/data-dashboards/hate-crime-dashboard/